In [ ]:
import gc
import numpy as np
import pandas as pd
import datatable as dt
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from tpot import TPOTClassifier

## September Tabular Playground Series - AutoML with Tpot

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64","object"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
def preprocess_data(df):
    num_missing = df.isna().sum(axis=1)
    # Fill missing values in training data with 0.0
    numerical_transformer = SimpleImputer(strategy='constant', fill_value=0.0)
    imputed_df1 = pd.DataFrame(numerical_transformer.fit_transform(df))
    imputed_df1.columns = df.columns
    
    df1 = reduce_memory_usage(imputed_df1)
    
    df1['missing'] = num_missing
    
    return df1

def make_sample(df, size=0.2):
    sampled_df = df.sample(int(len(df) * size))
    return sampled_df.drop(columns=['claim','id']), sampled_df['claim'].astype('int16')

In [ ]:
train_df = preprocess_data(dt.fread('../input/tabular-playground-series-sep-2021/train.csv').to_pandas())
test_df = preprocess_data(dt.fread('../input/tabular-playground-series-sep-2021/test.csv').to_pandas())

In [ ]:
test_df.id = test_df.id.astype('int32')

In [ ]:
# X holds the training data
# y holds the target/dependent variable, the 'claim' column.
# make_sample converts the 'claim' column to 'int8' since it is of type 'object'. tpot can't use object column as a target

X, y = make_sample(train_df, 0.1)

#### Garbage collecting

In [ ]:
del train_df
gc.collect()

Model Time

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, random_state=459)

In [ ]:
import multiprocessing
# This was taken from tpot documentation. It changes the fork behavior so that tpot uses less memory.
multiprocessing.set_start_method('forkserver')

pipeline_optimizer = TPOTClassifier(generations=20, population_size=50, scoring='roc_auc', memory='auto',
                                random_state=459, n_jobs=4, cv=2, verbosity=0) #config_dict='TPOT light'

In [ ]:
pipeline_optimizer.fit(X_train, y_train)

In [ ]:
pipeline_optimizer.export('exported_TPS_pipeline.py')

In [ ]:
roc_auc_score(y_valid, pipeline_optimizer.predict_proba(X_valid)[:,1])

In [ ]:
drop_id = test_df.drop(columns=['id'])

output = pd.DataFrame({
    'id': test_df.id,
    'claim': pipeline_optimizer.predict_proba(drop_id)[:,1]
})
output.to_csv('submission.csv', index=False)